In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os

notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'ensemble_commit' not found in notebook path.")


if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

import ensemble_model.preprocesser as preprocesser 
import ensemble_model.MoE_model_loss as moe 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer, DistilBertModel

# import whatthepatch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve



root
ensemble_commit
/root/ensemble_commit


In [2]:
moe.BiLSTMStudent

ensemble_model.MoE_model_loss.BiLSTMStudent

In [3]:
df = pd.read_json(r'../datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df

/tmp/ipykernel_1791/183603274.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,NA,NA,0,540958e2f5a87b81aa5f55ce40b3e2869754f97d,commit 540958e2f5a87b81aa5f55ce40b3e2869754f97...,diff --git a/drivers/staging/comedi/drivers/cb...,stoth68000,media-tree,wild
1,NA,NA,0,64d240b721b21e266ffde645ec965c3b6d1c551f,commit 64d240b721b21e266ffde645ec965c3b6d1c551...,diff --git a/drivers/target/target_core_file.c...,stoth68000,media-tree,wild
2,NA,NA,0,f181dd278274f50e689ebd13237010a90b430164,commit f181dd278274f50e689ebd13237010a90b43016...,diff --git a/include/paths.h b/include/paths.h...,openbsd,src,wild
3,NA,NA,0,0abdc3723b5d33dde698ab941325edec2819c128,commit 0abdc3723b5d33dde698ab941325edec2819c12...,diff --git a/gnu/usr.bin/binutils/ld/lexsup.c ...,openbsd,src,wild
4,NA,NA,0,d7930d7f820e5dd6b07b823f155aeb943b525e16,commit d7930d7f820e5dd6b07b823f155aeb943b525e1...,diff --git a/src/expat_erl.c b/src/expat_erl.c...,esl,MongooseIM,wild
...,...,...,...,...,...,...,...,...,...
35810,CVE-2013-0217,399,1,7d5145d8eb2b9791533ffe4dc003b129b9696c48,From 7d5145d8eb2b9791533ffe4dc003b129b9696c48 ...,diff --git a/drivers/net/xen-netback/netback.c...,torvalds,linux,cve
35811,CVE-2018-18311,119,1,34716e2a6ee2af96078d62b065b7785c001194be,From 34716e2a6ee2af96078d62b065b7785c001194be ...,diff --git a/util.c b/util.c\nindex 7282dd9cfe...,Perl,perl5,cve
35812,CVE-2019-12984,476,1,385097a3675749cbc9e97c085c0e5dfe4269ca51,From 385097a3675749cbc9e97c085c0e5dfe4269ca51 ...,diff --git a/net/nfc/netlink.c b/net/nfc/netli...,torvalds,linux,cve
35813,CVE-2013-0865,119,1,f3d16706060ab6ae6dc78f15359fab3fd87c9495,From f3d16706060ab6ae6dc78f15359fab3fd87c9495 ...,diff --git a/libavcodec/vqavideo.c b/libavcode...,NA,NA,cve


In [4]:
# df['command'] = df['diff'].apply(lambda x : extract_lines(find_longest_list([y.changes for y in whatthepatch.parse_patch(x)])))
# extracted_cols = df['diff'].str.extract(r'^diff --git (?P<source>[^\t\n]+) (?P<target>[^\t\n]+)')
# whatthepatch.apply_diff(diff, lao)

In [5]:
# len(df_dataset)

In [6]:
# df_dataset[0]

In [7]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('../models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

codebert_model = RobertaModel.from_pretrained('../models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('../models/codebert-base')

Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
# train_data = pd.read_csv('./datasets/PD_train.csv')
# test_data = pd.read_csv('./datasets/PD_test.csv')
# val_data = pd.read_csv('./datasets/PD_val.csv')



In [9]:
# codebert_model.config.hidden_size

In [10]:
len(test_data)

5372

In [11]:
train_data,_ = train_test_split(train_data, train_size=0.1, random_state=42)


In [12]:
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)

In [13]:
train_data['category'].value_counts()

category
0    1644
1     863
Name: count, dtype: int64

In [14]:
val_data

,index,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,31104,NA,NA,1,31e91e0f40547816f9256d5865d39d504706ad06,commit 31e91e0f40547816f9256d5865d39d504706ad0...,diff --git a/drivers/staging/most/aim-sound/so...,stoth68000,media-tree,wild
1,3833,NA,NA,0,dfea00191aec19dc1115934a3c06e97fd8338e2e,commit dfea00191aec19dc1115934a3c06e97fd8338e2...,diff --git a/drivers/media/video/mt9p031.c b/d...,mjg59,linux,wild
2,19412,NA,NA,0,21a36f4840b1f816364ca91e693619ae03266f08,commit 21a36f4840b1f816364ca91e693619ae03266f0...,diff --git a/cgminer.c b/cgminer.c\nindex 6e47...,sgminer-dev,sgminer,wild
3,11178,NA,NA,0,7547881d0951384f9833ec3a80fac8f3f16f3b98,commit 7547881d0951384f9833ec3a80fac8f3f16f3b9...,diff --git a/drivers/block/nvme.c b/drivers/bl...,stoth68000,media-tree,wild
4,13781,NA,NA,0,24adaf7958847e0950b197ac05cced2ea9e0b4e7,commit 24adaf7958847e0950b197ac05cced2ea9e0b4e...,diff --git a/drivers/iio/adc/ti_am335x_adc.c b...,mjg59,linux,wild
...,...,...,...,...,...,...,...,...,...,...
5368,21989,NA,NA,0,c4c373bab376f7d4accb8af8eb3dfc41a99ae2d4,commit c4c373bab376f7d4accb8af8eb3dfc41a99ae2d...,diff --git a/src/sbatch/sbatch.c b/src/sbatch/...,SchedMD,slurm,wild
5369,11040,NA,NA,0,8b7d2a4dc4dd285fdfb7a8e4f71c401c5e0a6566,commit 8b7d2a4dc4dd285fdfb7a8e4f71c401c5e0a656...,diff --git a/lib/asn1/der_get.c b/lib/asn1/der...,heimdal,heimdal,wild
5370,4840,NA,NA,0,d3d9acf646679c1981032b0985b386d12fccc60c,commit d3d9acf646679c1981032b0985b386d12fccc60...,diff --git a/kernel/trace/trace_events_filter....,mjg59,linux,wild
5371,15822,NA,NA,0,8379e4823d04e2552e1395c2410fc16733e28c6e,commit 8379e4823d04e2552e1395c2410fc16733e28c6...,diff --git a/drivers/gpu/drm/exynos/exynos_mix...,mjg59,linux,wild


In [15]:
val_data

,index,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,31104,NA,NA,1,31e91e0f40547816f9256d5865d39d504706ad06,commit 31e91e0f40547816f9256d5865d39d504706ad0...,diff --git a/drivers/staging/most/aim-sound/so...,stoth68000,media-tree,wild
1,3833,NA,NA,0,dfea00191aec19dc1115934a3c06e97fd8338e2e,commit dfea00191aec19dc1115934a3c06e97fd8338e2...,diff --git a/drivers/media/video/mt9p031.c b/d...,mjg59,linux,wild
2,19412,NA,NA,0,21a36f4840b1f816364ca91e693619ae03266f08,commit 21a36f4840b1f816364ca91e693619ae03266f0...,diff --git a/cgminer.c b/cgminer.c\nindex 6e47...,sgminer-dev,sgminer,wild
3,11178,NA,NA,0,7547881d0951384f9833ec3a80fac8f3f16f3b98,commit 7547881d0951384f9833ec3a80fac8f3f16f3b9...,diff --git a/drivers/block/nvme.c b/drivers/bl...,stoth68000,media-tree,wild
4,13781,NA,NA,0,24adaf7958847e0950b197ac05cced2ea9e0b4e7,commit 24adaf7958847e0950b197ac05cced2ea9e0b4e...,diff --git a/drivers/iio/adc/ti_am335x_adc.c b...,mjg59,linux,wild
...,...,...,...,...,...,...,...,...,...,...
5368,21989,NA,NA,0,c4c373bab376f7d4accb8af8eb3dfc41a99ae2d4,commit c4c373bab376f7d4accb8af8eb3dfc41a99ae2d...,diff --git a/src/sbatch/sbatch.c b/src/sbatch/...,SchedMD,slurm,wild
5369,11040,NA,NA,0,8b7d2a4dc4dd285fdfb7a8e4f71c401c5e0a6566,commit 8b7d2a4dc4dd285fdfb7a8e4f71c401c5e0a656...,diff --git a/lib/asn1/der_get.c b/lib/asn1/der...,heimdal,heimdal,wild
5370,4840,NA,NA,0,d3d9acf646679c1981032b0985b386d12fccc60c,commit d3d9acf646679c1981032b0985b386d12fccc60...,diff --git a/kernel/trace/trace_events_filter....,mjg59,linux,wild
5371,15822,NA,NA,0,8379e4823d04e2552e1395c2410fc16733e28c6e,commit 8379e4823d04e2552e1395c2410fc16733e28c6...,diff --git a/drivers/gpu/drm/exynos/exynos_mix...,mjg59,linux,wild


In [16]:


# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer, message='commit_message',command='diff_code',label='category')
val_dataset = preprocesser.SentencePairDataset(val_data, bert_tokenizer, codebert_tokenizer, message='commit_message',command='diff_code',label='category')
test_dataset = preprocesser.SentencePairDataset(test_data, bert_tokenizer, codebert_tokenizer, message='commit_message',command='diff_code',label='category')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)

In [17]:
base_model1 = moe.BaseModel(bert_model)
base_model2 = moe.BaseModel(codebert_model)

In [18]:
# Load BERT and CodeBERT models and tokenizers
bert_model = DistilBertModel.from_pretrained('/root/autodl-tmp/IPCK/model/distilbert')

codebert_model = RobertaModel.from_pretrained('../models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/IPCK/model/distilbert were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
encoder_model1 = moe.EncoderModel(bert_model)
encoder_model2 = moe.EncoderModel(codebert_model)

In [ ]:


# Create stacking model
moe_model = moe.MoEModel(base_model1, base_model2)
# Train the model
moe_model.trainer(train_loader, val_loader,num_epochs=30, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/30 Loss: 0.1970: 100%|██████████| 314/314 [01:51<00:00,  2.81batch/s]


=============================train========================


Epoch 2/30 Loss: 0.1884: 100%|██████████| 314/314 [01:44<00:00,  3.02batch/s]


=============================train========================


Epoch 3/30 Loss: 0.1662: 100%|██████████| 314/314 [01:48<00:00,  2.88batch/s]


...

Epoch 7/30 Loss: 0.1160: 100%|██████████| 314/314 [01:52<00:00,  2.78batch/s]


=============================train========================


Epoch 8/30 Loss: 0.1196: 100%|██████████| 314/314 [01:52<00:00,  2.78batch/s]


=============================train========================


Epoch 9/30:  93%|█████████▎| 292/314 [01:46<00:05,  3.99batch/s]

In [ ]:
# test_dataset = preprocesser.SentencePairDataset(test_data, bert_tokenizer, codebert_tokenizer,message='commit_message',command='diff_code',label='category')
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [24]:
torch.save(moe_model, "entire_model.pth")

In [20]:
moe_model = torch.load("entire_model.pth")                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [25]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = moe_model.evaluate(test_loader)

Validation Accuracy: 0.8997
Precision: 0.8990
Recall: 0.8997
F1-Score: 0.8987


In [27]:
test_acc

0.8883097542814594

In [16]:
model_save_path = 'best_model'

In [25]:
student_model = moe.BiLSTMStudent(hidden_dim=256,output_dim =2, base_model1=encoder_model1,base_model2=encoder_model2)

In [22]:
# train_student_model

In [26]:
student_model.distill_trainer(moe_model, test_loader, num_epochs=1)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/1 Loss: 0.9975: 100%|██████████| 672/672 [03:33<00:00,  3.15batch/s]


Epoch 1/1, Training Loss: 0.9975
Validation Accuracy: 0.8987
Precision: 0.9009
Recall: 0.8987
F1-Score: 0.8961


In [27]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = student_model.evaluate(val_loader)

Validation Accuracy: 0.9025
Precision: 0.9053
Recall: 0.9025
F1-Score: 0.8994


In [28]:
torch.save(student_model, "student_model.pth")